# 분석 및 모델링 프로세스

1. 피쳐 데이터 타입들 살펴보기
- 1차 차원 축소: 값이 하나 혹은 없는 칼럼 및 도메인 지식상 무의미한 칼럼 삭제
- 칼럼별 유실 데이터 보기
- 호스트설명텍스트피쳐/사용자리뷰텍스트/디테일피쳐 분리
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 피쳐 이해하기
    - 분포/skewnness 로그 적용 할지 안 할지
    - numeric or categorical or text
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- Missing Data Imputation (독립변수간 회귀)
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 다중공선성 확인 (피쳐간 독립성 확인)
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 피쳐 추가 및 변형
    - 중심가로부터의 거리
    - 가장 가까운 지하철 로부터의 거리
    - transit 관련 설명 유무
    - 설명에 단어수로 인한 친절도?
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 다중공선성(VIF)/F-검정을 통한 차원 축소 
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 정규화 적용 (릿지, 라쏘, 엘라스틱넷)
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 아웃라이어 확인 및 데이터 확인후 제거
- 피팅 및 교차 검증, 잔차 정규성 테스트, 조건수 확인, QQ Plot 그리고 Residual Plot
- 다중회귀 피팅

In [7]:
pd.options.display.max_columns = 300

# 1. 피쳐 데이터 타입들 살펴보기

In [2]:
df_listings = pd.read_csv('./data/train.csv')
df_listings.tail(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
30468,30471,2015-06-30,45,NaN,10.0,20.0,1.0,NaN,1.0,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30469,30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,22,1,1,6,31,1,4,65,7,13500000
30470,30473,2015-06-30,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,...,5,2,0,7,16,0,9,54,10,5600000


In [3]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Columns: 292 entries, id to price_doc
dtypes: float64(119), int64(157), object(16)
memory usage: 67.9+ MB


In [4]:
df_listings.shape

(30471, 292)

# 2. 1차 차원 축소: 값이 하나 혹은 없는 칼럼 및 도메인 지식상 무의미한 칼럼 삭제

In [8]:
df_listings.nunique()

id                                       30471
timestamp                                 1161
full_sq                                    211
life_sq                                    175
floor                                       41
max_floor                                   49
material                                     6
build_year                                 119
num_room                                    13
kitch_sq                                    74
state                                        5
product_type                                 2
sub_area                                   146
area_m                                     146
raion_popul                                146
green_zone_part                            146
indust_part                                132
children_preschool                         146
preschool_quota                            121
preschool_education_centers_raion           13
children_school                            146
school_quota 

In [9]:
# 값이 하나도 없거나 하나인 칼럼들 골라내기
cols_to_drop = list(df_listings.nunique()[lambda col : (col==1) | (col==0)].index)
cols_to_drop

[]

In [11]:
df_listings.tail(1000)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [12]:
df_listings['timestamp'].unique()

array(['2011-08-20', '2011-08-23', '2011-08-27', ..., '2015-06-27',
       '2015-06-29', '2015-06-30'], dtype=object)

In [14]:
df_listings['product_type'].unique()

array(['Investment', 'OwnerOccupier'], dtype=object)

In [17]:
df_listings['sub_area'].unique()

array(['Bibirevo', 'Nagatinskij Zaton', "Tekstil'shhiki", 'Mitino',
       'Basmannoe', 'Nizhegorodskoe', "Sokol'niki", 'Koptevo', 'Kuncevo',
       'Kosino-Uhtomskoe', 'Zapadnoe Degunino', 'Presnenskoe',
       'Lefortovo', "Mar'ino", "Kuz'minki", 'Nagornoe', "Gol'janovo",
       'Vnukovo', 'Juzhnoe Tushino', 'Severnoe Tushino',
       "Chertanovo Central'noe", 'Fili Davydkovo', 'Otradnoe',
       'Novo-Peredelkino', 'Bogorodskoe', 'Jaroslavskoe', 'Strogino',
       'Hovrino', "Moskvorech'e-Saburovo", 'Staroe Krjukovo', 'Ljublino',
       'Caricyno', 'Veshnjaki', 'Danilovskoe', 'Preobrazhenskoe',
       "Kon'kovo", 'Brateevo', 'Vostochnoe Izmajlovo', 'Vyhino-Zhulebino',
       'Donskoe', 'Novogireevo', 'Juzhnoe Butovo', 'Sokol', 'Kurkino',
       'Izmajlovo', 'Severnoe Medvedkovo', 'Rostokino',
       'Orehovo-Borisovo Severnoe', 'Ochakovo-Matveevskoe', 'Taganskoe',
       'Dmitrovskoe', 'Orehovo-Borisovo Juzhnoe', 'Teplyj Stan',
       'Babushkinskoe', 'Pokrovskoe Streshnevo', 'Obruc

In [20]:
len(df_listings['ID_metro'].unique())

223

In [22]:
df_listings['cafe_count_500_price_high'].unique()

array([0, 1, 2, 3])

In [21]:
df_listings['ecology'].unique()

array(['good', 'excellent', 'poor', 'satisfactory', 'no data'],
      dtype=object)

# column 정리
- id : 아이디
- timestamp : 년월일 2011년~2015
- full_sq : 전체 면적?
- life_sq : 라이프 면적?
- floor : 층 수
- max_floor : 최고층
- material : 재료? (int)
- bulid_year : 지어진 년도
- num_room : 방 갯수
- kitch_sq : 주방 면적
- state : ??
- product_type : 매물 종류(자가/투자) -> 카테고리로 변경할 수 있음
- sub_area : District 구역 (146개) -> 중복되는게 몇개 보임, 쓰는 방법이 달라서 그런 것 같다.
- area_m : 면적
- raion_popul : Number of municipality population. district
- green_zone_part : 그린존 구역
- indust_part : 산업구역
- children_preschool : 유치원
- preschool_quota : Number of seats in pre-school(유치원) organizations
- preschool_education_centers_raion : Number of pre-school institutions
- school_education_centers_top_20_raion : Number of high schools of the top 20 best schools in Moscow
- hospital_beds_raion : 병원 침대수??
- healthcare_centers_raion : Number of healthcare centers in district
- university_top_20_raion : Number of higher education institutions in the top ten ranking of the Federal rank
- sport_objects_raion : 스포츠센터 수
- additional_education_raion : 추가 교육기관 수
- culture_objects_top_25 : 문화센터 top 25 (yes/no) -> 카테고리로 변경할 수 있음
- culture_objects_top_25_raion : Number of objects of cultural heritage
- shopping_centers_raion : 쇼핑센터 수
- office_raion : 오피스 수
- thermal_power_plant_raion : Distance to thermal power plant(화력발전소와 거리) (yes/no) -> 카테고리로 변경할 수 있음
- incineration_raion : 소각장과 거리 , (yes/no) -> 카테고리로 변경할 수 있음
- oil_chemistry_raion : 오일화학장과 거리, (yes/no) -> 카테고리로 변경할 수 있음
- radiation_raion : 원자력발전소와 거리, (yes/no) -> 카테고리로 변경할 수 있음
- railroad_terminal_raion : 기차터미널과 거리, (yes/no) -> 카테고리로 변경할 수 있음
- big_market_raion : 대형마트와 거리, (yes/no) -> 카테고리로 변경할 수 있음
- nuclear_reactor_raion : 핵발전소와 거리, (yes/no) -> 카테고리로 변경할 수 있음
- detention_facility_raion : 교도소와 거리, (yes/no) -> 카테고리로 변경할 수 있음
- full_all : Total number of population in the municipality(지방) : 전체 남녀 인구 수
- male_f : 남성 수
- female_f : 여성 수
- young_all : 젋은 전체 인구 수
- young_male : 젋은 남자 인구 수
- young_female : 젋은 여자 인구 수
- work_all : 전체 직장인 남녀 인구 수
- work_male : 직장인 남자 인구 수
- work_female : 직장인 여자 인구 수
- ekder_all : 노인 전체 인구 수
- ekder_male : 남자 노인 수
- ekder_female : 여자 노인 수
- 0_6_all : 0_6 전체 수???
- 0_6_male : 0_6 남성 수
- 0_6_female : 0_6 여성 수
- 7_14_all : 7_14 전체 수
- 7_14_male : 7_14 남성 수
- 7_14_female : 7_14 여성 수
- 0_17_all : 0_17 전체 수
- 0_17_male : 0_17 남성 수
- 0_17_female : 0_17 여성 수
- 16_29_all : 16_29 전체 수
- 16_29_male : 16_29 남성 수
- 16_29_female : 16_29 여성 수
- 0_13_all : 0_13 전체 수 
- 0_13_male : 0_13 남성 수
- 0_13_female : 0_13 여성 수
- raion_build_count_with_material_info : 건물 갯수
- build_count_block : 블럭건물 수
- build_count_wood : 나무건물 수
- build_count_frame : 프레임 건물 수
- build_count_brick : 벽돌건물 수
- build_count_monolith : monolith 건물 수
- build_count_panel : panel 건물 수
- build_count_foam : foam 건물 수
- build_count_slag : slag(화산) 건물 수
- build_count_mix : mix 건물 수
- raion_build_count_with_builddate_info : 건물지은년도 정보가 있는 건물 수
- build_count_before_1920 : 1920년도 전에 지은 건물 수
- build_count_1921-1945 : 1921-1945 건물 수
- build_count_1946-1970 : 1946-1970 건물 수
- build_count_1971-1995 : 1971-1995 건물 수
- build_count_after_1995 : 1995년 이후 건물 수
- ID_metro 
- metro_min_avto : metro까지 분?
- metro_km_avto : metro까지 km?
- metro_min_walk : 메트로까지 걸어서 걸린 시간(분)
- metro_km_walk : 매트로까지 걸어서 걸린 거리
- kindergarten_km : kindergarten 거리
- school_km : 학교 거리
- park_km : 공원 거리
- green_zone_km : green_zone 거리
- industrial_km : 산업장 거리
- water_treatment_km : water_treatment 거리
- cemetery_km : cemetery 거리
- incineration_km : incineration(소각) 거리
- railroad_station_walk_km : 걸어서 지하철역까지 거리
- railroad_station_walk_min : 걸어서 지하철역까지 걸리는 시간
- ID_railroad_station_walk : 걸어서 가는 역 ID
- railroad_station_avto_km : 기차역까지 거리
- railroad_station_avto_min : 기차역까지 걸리는 시간
- ID_railroad_station_avto : 자동차 기차역 ID
- public_transport_station_km : 공공 교통역까지 거리
- public_transport_station_min_walk : 공공 교통역까지 걸리는 시간
- water_km : 강과의 거리
- water_1line : 강 1line? (yes/no) -> 카테고리로 변경할 수 있음
- mkad_km : Distance to MKAD (Moscow Circle Auto Road)
- ttk_km : 전화국까지 거리
- sadovoe_km : garden ring과의 거리
- bulvar_ring_km : bulvar_ring(비지니스거리?) 과의 거리
- kremlin_km : 크렘린과의 거리
- big_road1_km : 큰도로와의 거리
- ID_big_road1 
- big_road1_1line : big_road1_1line 여부 (yes/no) -> 카테고리로 변경할 수 있음
- big_road2_km : big_road2 와의 거리
- ID_big_road2
- railroad_km : 기차길 거리
- railroad_1line : railroad_1line 여부 (yes/no) -> 카테고리로 변경할 수 있음
- zd_vokzaly_avto_km : zd_vokzaly 차타고 거리
- ID_railroad_terminal : 기차길터미널 ID
- bus_terminal_avto_km : 버스터미널까지 차타고 거리
- ID_bus_terminal : 버스터미널 ID
- oil_chemistry_km : 오일화학까지 거리
- nuclear_reactor_km : 핵발전소까지 거리
- radiation_km : 원자력까지 거리
- power_transmission_line_km : power_transmission_line 거리
- thermal_power_plant_km : thermal_power_plant 거리
- ts_km : Distance to power station
- big_market_km : Distance to big_market
- market_shop_km : Distance to market_shop
- fitness_km : Distance to fitness 
- swim_pool_km : Distance to swim_pool
- ice_rink_km : Distance to ice_rink
- stadium_km : Distance to stadium
- basketball_km : Distance to basketball
- hospice_morgue_km : Distance to hospice_morgue(시체안치소)
- detention_facility_km : 유치장과의 거리
- public_healthcare_km : 공공체육시설과 거리
- university_km : 대학과의 거리
- workplaces_km : workplaces와의 거리
- shopping_centers_km : 쇼핑센터와의 거리
- office_km : 오피스와의 거리
- additional_education_km : 추가교육기관 거리
- preschool_km : 유치원과 거리
- big_church_km : 큰교회와 거리
- church_synagogue_km : 유대교와 거리
- mosque_km : 시체안치소와 거리
- exhibition_km : 전시장과 거리
- catering_km : catering과의 거리
- ecology : 생태학 ('good', 'excellent', 'poor', 'satisfactory', 'no data') -> 카테고리로 변경
- green_part_500 : The share of green zones in 500 meters zone
- prom_part_500 : The share of industrial(산업) zones in 500 meters zone
- office_count_500 : The number of office space in 500 meters zone
- trc_count_500 : The number of shopping malls(쇼핑몰) in 500 meters zone
- trc_sqm_500 : The square of shopping malls in 500 meters zone
- cafe_count_500 : The number of cafe in 500 meters zone
- cafe_sum_500_min_price_avg : 카페 평균 최저 가격이 500미터 안에 총 카페 수
- cafe_sum_500_max_price_avg : 카페 평균 최고 가격이 500미터 안에 총 카페 수
- cafe_avg_price_500 : 500미터 안 카페 평균가격
- cafe_count_500_na_price = Cafes bill N/A in 500 meters zone
- cafe_count_500_price_500 : Cafes and restaurant bill, average under 500 in 500 meters zone
- cafe_count_500_price_1000 : Cafes and restaurant bill, average under 500 in 1000 meters zone
- cafe_count_500_price_1500 : Cafes and restaurant bill, average under 500 in 1500 meters zone
- cafe_count_500_price_2500 : Cafes and restaurant bill, average under 500 in 2500 meters zone
- cafe_count_500_price_4000 : Cafes and restaurant bill, average under 500 in 4000 meters zone
- cafe_count_500_price_high : Cafes and restaurant bill, average under 500 in high meters zone, (0, 1, 2, 3)
- big_church_count_500 : The number of big_church in 500 meters zone
- church_count_500 : The number of church in 500 meters zone , 존재여부(Yes/No로 카테고리화???)
- mosque_count_500 : The number of mosque in 500 meters zone
- leisure_count_500 : The number of leisure in 500 meters zone
- sport_count_500 : The number of sport in 500 meters zone
- market_count_500 : The number of market in 500 meters zone
- green_part_1000 : The number of green_part in 1000 meters zone
- 